Import libraries

In [23]:
from bs4 import BeautifulSoup
import requests
import pyperclip
import wikipedia
from pathlib import Path

Load edible.txt file: edible mushrooms according to wikipedia (https://pl.wikipedia.org/wiki/Grzyby_jadalne)

In [181]:
edible = Path('03_edible_mushrooms_wiki.txt').read_text(encoding = 'utf8')

In [182]:
edible

'=== Lista Ministerstwa Zdrowia RP ===\nZałącznik z 16 czerwca 2020 do Rozporządzenia Ministra Zdrowia z dnia 17 maja 2011.<ref>\'\'Rozporządzenie Ministra Zdrowia z dnia 13 listopada 2008 r. w sprawie grzybów dopuszczonych do obrotu lub produkcji przetworów grzybowych oraz środków spożywczych zawierających grzyby oraz uprawnień klasyfikatora grzybów i grzyboznawcy\'\' ({{Dziennik Ustaw|2020|1048}}).</ref> zawiera grzyby zarówno rosnące dziko na terenie Polski, jak i uprawiane lub importowane:\n* [[boczniak ostrygowaty]] (\'\'Pleurotus ostreatus\'\' (Jacq.) Kummer)\n* [[borowik szlachetny]] (prawdziwek wszystkie odmiany) (\'\'Boletus edulis\'\' Bull ex Fr.)\n* [[czubajka kania]] (\'\'Macrolepiota procera\'\' (Soop.) Sing)\n* [[gąska zielonka]] (\'\'Tricholoma flavovirens\'\' (Pers.) Lund ex Nannf)\n* [[kolczak obłączasty]] (\'\'Hydnum repandum\'\' L.)\n* [[koźlarz babka]] (wszystkie odmiany) (\'\'Leccinum scabrum\'\' (Bull) S. F. Gray.)\n* [[koźlarz czerwony]] (wszystkie odmiany) (\'\'

Create 2 lists:
* mushroom names (in format 'Amanita_echinocephala')
* mushroom webpages (in format 'https://grzyby.pl/pelna/gatunki/Amanita_echinocephala')

In [69]:
# Credentials to use full page functionality
credentials = dict(l.rstrip().split('=') for l in open('config.properties') if not l.startswith("#"))
URL_init = "https://www.grzyby.pl/logowanie"

# Log in using credential
form_data = credentials
s = requests.Session()
server = s.post(URL_init, data = form_data)

# Get full list of species
URL = 'https://www.grzyby.pl/pelna/gatunki/'
page = s.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

mushroom_names = []
mushroom_webs = []
for item in soup.find_all('td'):
    if item.find('a', href = True) and '_' in str(item):
        name = item.find('a').text
        name_clean = item.find('a').text.replace('.htm', '')
        if len(name_clean.split('_')) == 2:
            mushroom_webs.append('https://grzyby.pl/pelna/gatunki/'+name)
            mushroom_names.append(name_clean)

# Change 'muchomor jezowaty' to 'Amanita_echinocephala'
mushroom_names[-1] = 'Amanita_echinocephala'

Create mushroom dictionary

In [183]:
# Empty dictionary
mushroom_dict = dict()

# Use mushroom_names and mushroom_webs to update keys & values
mushroom_dict = {mushroom_names[i]: {'edible': None, 'web': mushroom_webs[i], 'img' : []} for i in range(len(mushroom_names))}

# Change 'edible' value
for key, value in mushroom_dict.items():
    if str(key).replace('_',' ') in edible:
        value['edible'] = 'e'
    else:
        value['edible'] = 'p'

Create short list for further tests

In [184]:
# Create mushroom short list - TEST
mushroom_names_short = mushroom_names[430:440]
mushroom_names_short

['Boletopsis_leucomelaena',
 'Boletus_aereus',
 'Boletus_appendiculatus',
 'Boletus_betulicola',
 'Boletus_calopus',
 'Boletus_carpinaceus',
 'Boletus_depilatus',
 'Boletus_edulis',
 'Boletus_erythropus',
 'Boletus_fechtneri']

In [185]:
# Create mushroom dictionary - TEST
mushroom_dict_short = dict()
mushroom_dict_short = {mushroom_names_short[i]: {'edible': None, 'web': mushroom_webs[i+430], 'img' : []} for i in range(len(mushroom_names_short))}

# Change 'edible' value
for key, value in mushroom_dict_short.items():
    if str(key).replace('_',' ') in edible:
        value['edible'] = 'e'
    else:
        value['edible'] = 'p'

In [187]:
# Check - TEST
mushroom_dict_short

{'Boletopsis_leucomelaena': {'edible': 'p',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletopsis_leucomelaena.htm',
  'img': []},
 'Boletus_aereus': {'edible': 'e',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_aereus.htm',
  'img': []},
 'Boletus_appendiculatus': {'edible': 'e',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_appendiculatus.htm',
  'img': []},
 'Boletus_betulicola': {'edible': 'p',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_betulicola.htm',
  'img': []},
 'Boletus_calopus': {'edible': 'e',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_calopus.htm',
  'img': []},
 'Boletus_carpinaceus': {'edible': 'p',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_carpinaceus.htm',
  'img': []},
 'Boletus_depilatus': {'edible': 'p',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_depilatus.htm',
  'img': []},
 'Boletus_edulis': {'edible': 'e',
  'web': 'https://grzyby.pl/pelna/gatunki/Boletus_edulis.htm',
  'img': []},
 'Boletus_erythropus': {'edible': 'e',
  'we

In [171]:
# Get all images sources from website
# URL = "https://www.grzyby.pl/pelna/gatunki/Psilocybe_semilanceata.htm"
# page = s.get(URL)
# soup = BeautifulSoup(page.content, "html.parser")
#
# for item in soup.find_all('img'):
#     if 'Psilocybe semilanceata' in str(item['alt']) and 'mapa'not in str(item['src']) and 'icon'not in str(item['src']):
#         print('https://www.grzyby.pl'+item['src'])

https://www.grzyby.pl/pelna/foto/03/030928-6598.jpg
https://www.grzyby.pl/pelna/foto/04/041006-9568.jpg
https://www.grzyby.pl/pelna/foto/03/030928-6484is.jpg
https://www.grzyby.pl/pelna/foto/mj/mj-PsilocybeSemilanceolata-8050is.jpg
https://www.grzyby.pl/pelna/foto/04/041006-9570is.jpg
https://www.grzyby.pl/pelna/foto/03/040314-6952is.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221665ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221666ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221667ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221668ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221669ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221670ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221671ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221672ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-122993-221673ss.jpg
https://www.grzyby.pl/pelna/foto/GREJ/GREJ-145348-274539ss.jpg
https://www.grzyby.pl/pelna/fo

In [216]:
# Get latin name from website
# >> sometimes there is mismatch between page name and latin name/title

URL = "https://www.grzyby.pl/pelna/gatunki/Psilocybe_semilanceata.htm"
page = s.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

latin_name = ' '.join(soup.find('title').text.split(' ')[0:2])

In [217]:
# Add images to dictionary - TEST
# Credentials
credentials = dict(l.rstrip().split('=') for l in open('config.properties') if not l.startswith("#"))
URL_init = "https://www.grzyby.pl/logowanie"

# Log in using credential
form_data = credentials
s = requests.Session()
server = s.post(URL_init, data = form_data)

# Add images to dict
for key, value in mushroom_dict_short.items():
    URL = value['web']
    page = s.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    latin_name = ' '.join(soup.find('title').text.split(' ')[0:2])

    for item in soup.find_all('img'):
        if latin_name in str(item['alt']) and 'mapa'not in str(item['src']) and 'icon'not in str(item['src']):
            value['img'].append(('https://www.grzyby.pl'+item['src']))

In [219]:
# Check
for key, value in mushroom_dict_short.items():
    if len(value['img']) != 0:
        print(key)
        print(value['edible'])
        print(value['web'])
        print(value['img'])

Boletopsis_leucomelaena
p
https://grzyby.pl/pelna/gatunki/Boletopsis_leucomelaena.htm
['https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276439ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276440ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276441ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276442ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276443ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-184257-402936ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-184257-402937ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-184257-402938ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-184257-402939ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276439ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276440ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276441ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-276442ss.jpg', 'https://www.grzyby.pl/pelna/foto/GREJ/GREJ-146176-2764